In [ ]:
import requests
import pandas as pd

In [ ]:
def fetch_restaurants(api_key, location='San Francisco', term='restaurants', limit=50, output_file='sf_restaurants.csv'):
    headers = {
        'Authorization': f'Bearer {api_key}'
    }

    params = {
        'location': location,
        'term': term,
        'limit': limit,
        'offset': 0
    }

    all_businesses = []

    response = requests.get('https://api.yelp.com/v3/businesses/search', headers=headers, params=params)

    # Print the full response for debugging
    print("Response Status Code:", response.status_code)
    if response.status_code == 200:
        data = response.json()
        print("Response Data:", data)  # Print the full response data

        if 'businesses' in data:
            total_results = data['total']
            results_per_page = 50

            for offset in range(0, total_results, results_per_page):
                params['offset'] = offset
                response = requests.get('https://api.yelp.com/v3/businesses/search', headers=headers, params=params)
                data = response.json()

                if 'businesses' in data:
                    all_businesses.extend(data['businesses'])
                else:
                    print("No businesses found in the response at offset", offset)
        else:
            print("No businesses found in the initial response.")
    else:
        print("Error:", response.status_code)
        print(response.json())

    if all_businesses:
        # Extract relevant data
        business_data = []

        for business in all_businesses:
            business_info = {
                'id': business.get('id'),
                'name': business.get('name'),
                'rating': business.get('rating'),
                'review_count': business.get('review_count'),
                'price': business.get('price'),
                'categories': [category['title'] for category in business.get('categories', [])],
                'address': ", ".join(business['location']['display_address']),
                'latitude': business['coordinates']['latitude'],
                'longitude': business['coordinates']['longitude'],
                'phone': business.get('display_phone'),
                'url': business.get('url')
            }
            business_data.append(business_info)

        # Create a DataFrame and save to CSV
        df = pd.DataFrame(business_data)
        df.to_csv(output_file, index=False)
        print(f"Data saved to {output_file}")
    else:
        print("No business data collected.")

# Call the function with your API key
API_KEY = 'n5PtWi9IO6DU7C_ZtPLS7AeKcnP-kzJCT23gZDlTBxfI-buC7is4Is0e3wxNdyS084O2i0n88J3_RNDJ6CNVtSew1GHkp_yZgWE7xIP0IGa8-nC98IpAaR_0Lr5kZnYx'
fetch_restaurants(API_KEY)